In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import itertools

import warnings
warnings.filterwarnings("ignore")

In [4]:
#################################################
#### ClueGO cyREST Python 3 Example Workflow ####
#################################################

# In Windows install first python for Windows 3.7.2 from https://www.python.org/downloads/release/python-372/
#### Please make sure that Cytoscape v3.6+' is started and the Cytoscape Apps 'yFiles Layout Algorithms' and 'ClueGO v2.5.2' are installed before running this script! ####

# required libraries that need to be installed before starting the script
# In Windows install first pip with e.g. py get-pip.py  (https://bootstrap.pypa.io/get-pip.py)
import requests # to install use "pip install requests" or "sudo apt install python3-request" in ubuntu linux
import json
import os
import csv
from pathlib import Path
from urllib.parse import quote
import time

# Helper functions
def read_gene_list(data):
    gene_ids=[]
    with open(data,'r') as f:
        reader=csv.reader(f,delimiter='\t')
        for row in reader:
            gene_ids.append(row[0])
    return gene_ids

def writeLines(lines,out_file):
    file = open(out_file,'w')
    for line in lines:
        file.write(line)
    file.close()

def writeBin(raw,out_file):
    file = open(out_file,'wb')
    file.write(raw)
    file.close()

def dframe_stack_list(ins,col,typ='float32'):
    df = ins.copy(deep=True)
    s = df[col].apply(lambda x: pd.Series(list(x)),1).stack().astype(typ)
    s.index = s.index.droplevel(-1)
    s.name = col
    del df[col]
    df = df.join(s)
    return df

def fix_pvals(x):
    if x < 1e-16:
        x=1e-16
    return x

In [11]:
#### Basic settings for cyREST ####
# home.folder = "/home/user"
EXAMPLE_SELECTION = "ClueGO_output"
HOME_FOLDER = str(Path.home())

# # Windows specific command to set home folder to user-home
# HOME_FOLDER = HOME_FOLDER.replace("\\Documents","")

SEP = "/"
OUTPUT_FOLDER = HOME_FOLDER+SEP+EXAMPLE_SELECTION
try:
    os.stat(OUTPUT_FOLDER)
except:
    os.mkdir(OUTPUT_FOLDER)

CLUEGO_HOME_FOLDER = HOME_FOLDER+SEP+"ClueGOConfiguration"+SEP+"v2.5.7"
PORT_NUMBER = "1234"
HOST_ADDRESS = "localhost"
HEADERS = {'Content-Type': 'application/json'}

# define base urls
CYTOSCAPE_BASE_URL = "http://"+HOST_ADDRESS+":"+PORT_NUMBER+SEP+"v1"
CLUEGO_BASE_URL = CYTOSCAPE_BASE_URL+SEP+"apps"+SEP+"cluego"+SEP+"cluego-manager"

print("Example analysis Parameters for "+EXAMPLE_SELECTION)
print("User Home Folder: "+HOME_FOLDER)
print("User Output Folder: "+OUTPUT_FOLDER)
print("Cytoscape Base URL: "+CYTOSCAPE_BASE_URL)
print("ClueGO Base URL: "+CLUEGO_BASE_URL)


Example analysis Parameters for ClueGO_output
User Home Folder: /Users/zrimec
User Output Folder: /Users/zrimec/ClueGO_output
Cytoscape Base URL: http://localhost:1234/v1
ClueGO Base URL: http://localhost:1234/v1/apps/cluego/cluego-manager


### Load data

In [13]:
# read in all the data
# append from different sheets to one df

sheet_names = ['acetate',
          'furfural',
          'HMF',
          'formic acid',
          'phenolics',
          'hydrolysates']

df = []
for i in range(6):
    df.append(pd.read_excel('../data/210310_List target genes_EC.xlsx',sheet_name=i))
    df[-1]['sheet'] = sheet_names[i]
    if i==0:
        df[-1] = df[-1].drop(0)
df = pd.concat(df)
# fix col names
df.columns = pd.Series(df.columns).str.lower().str.replace(' ','_')
print(df.shape)
df.head()

# combine groups
df['sheet'] = df.sheet.map({'acetate':'ac_for',
                  'formic acid':'ac_for',
                  'HMF':'fur_HMF',
                  'furfural':'fur_HMF',
                  'hydrolysates':'hydrolysates',
                  'phenolics':'phenolics'
                 })
df.groupby('sheet').gene.count()
df.groupby('sheet')['strain','conditions'].nunique()


(7020, 13)


,strain,conditions
sheet,,
ac_for,18,39
fur_HMF,12,18
hydrolysates,17,19
phenolics,6,16


In [14]:
# Load yeast gene names and GO slims
# folder = '../../DeepExpression/paper-z1/scripts/'
# fn1 = 'SGD_features.tab'

import wget
url = 'https://downloads.yeastgenome.org/curation/chromosomal_feature/SGD_features.tab'
fname = wget.download(url)
columns = ['Primary_SGDID',
           'Feature_type',
            'Feature_qualifier',
            'Feature_name',
            'Standard_gene_name',
            'Alias', # (optional, multiples separated by |)
            'Parent_feature_name',
            'Secondary_SGDID',
            'Chromosome',
            'Start_coordinate',
            'Stop_coordinate',
            'Strand',
            'Genetic_position',
            'Coordinate_version',
            'Sequence_version',
            'Description']
gene_function = (pd
                 .read_csv(fname,sep='\t',header=None,names=columns)
                 #.set_index('Feature name')
                 .query('Feature_type=="ORF"')
                )
print(gene_function.shape)

print(sum(df.gene.isin(gene_function.Alias.unique()))/df.shape[0])
print(sum(df.gene.isin(gene_function.Feature_name.unique()))/df.shape[0])
print(sum(df.gene.isin(gene_function.Standard_gene_name.unique()))/df.shape[0])

gene_function.head()

(6604, 16)
0.0065527065527065526
0.1284900284900285
0.8021367521367522


,Primary_SGDID,Feature_type,Feature_qualifier,Feature_name,Standard_gene_name,Alias,Parent_feature_name,Secondary_SGDID,Chromosome,Start_coordinate,Stop_coordinate,Strand,Genetic_position,Coordinate_version,Sequence_version,Description
0,S000002143,ORF,Dubious,YAL069W,NaN,NaN,chromosome 1,NaN,1,335.0,649.0,W,NaN,1996-07-31,1996-07-31,Dubious open reading frame; unlikely to encode...
2,S000028594,ORF,Dubious,YAL068W-A,NaN,NaN,chromosome 1,NaN,1,538.0,792.0,W,NaN,2003-07-29,2003-07-29,Dubious open reading frame; unlikely to encode...
9,S000002142,ORF,Verified,YAL068C,PAU8,seripauperin PAU8,chromosome 1,NaN,1,2169.0,1807.0,C,NaN,1996-07-31,1996-07-31,Protein of unknown function; member of the ser...
11,S000028593,ORF,Uncharacterized,YAL067W-A,NaN,NaN,chromosome 1,NaN,1,2480.0,2707.0,W,NaN,2003-07-29,2003-07-29,Putative protein of unknown function; identifi...
14,S000000062,ORF,Verified,YAL067C,SEO1,putative permease SEO1,chromosome 1,L000003363,1,9016.0,7235.0,C,NaN,2011-02-03,1996-07-31,Putative permease; member of the allantoate tr...


In [15]:
# check gene coverage
# are there gene names that are not standard?
df[~df.gene.isin(gene_function.Standard_gene_name.unique())].gene

# combine all name types
all_names=list(gene_function.Alias.unique())
all_names.extend(list(gene_function.Feature_name.unique()))
all_names.extend(list(gene_function.Standard_gene_name.unique()))
len(all_names)
print(sum(df.gene.isin(all_names))/df.shape[0])
df[~df.gene.isin(all_names)].gene

# fix gene names to capital final letters as it is in SGD
# fix italic gene names => remove \xa0 string
# remove spaces
df['gene_fix'] = (df.gene
                  .astype(str)
                  .apply(lambda x: x.replace('\xa0','').replace(' ',''))
                  .apply(lambda x: x.upper())
                 )
df.gene_fix

# then recheck how many
print(sum(df.gene_fix.isin(all_names))/df.shape[0])
df[~df.gene_fix.isin(all_names)].gene_fix.tolist()

df['gene'] = df.gene_fix

0.935042735042735
0.9884615384615385


In [17]:
# prepare overlapping gene lists

gene_lists = []
for inh1,inh2 in list(itertools.combinations(df.sheet.unique(),2)):
    for eff in ['+','-']:
        if inh1!=inh2:
            # prepare test set
            sel1 = (df
                 .query('sheet==@inh1')
                 .query('growth_effect==@eff'))
            sel2 = (df
                 .query('sheet==@inh2')
                 .query('growth_effect==@eff'))
            gene_lists.append([inh1,inh2,eff,pd.DataFrame(sel1[sel1.gene.isin(sel2.gene)]
                        .gene.drop_duplicates())
                        #.merge(df_go,on='gene',how='left')
                        #.dropna(subset=['gene','entrez_id'])
                        #.entrez_id
                        .gene
                        .tolist()])
gene_lists = pd.DataFrame(gene_lists,columns=['inh1','inh2','effect','names'])
gene_lists

,inh1,inh2,effect,names
0,ac_for,fur_HMF,+,"[ACE2, CKB2, ECM30, GIS4, GRE3, GTR2, HAL5, HE..."
1,ac_for,fur_HMF,-,"[AIM10, AIM4, ARF1, BCH1, BST1, BUD28, CAT8, C..."
2,ac_for,phenolics,+,"[ADE1, ADE13, ADE17, ADR1, ALD5, ATF1, ATF2, B..."
3,ac_for,phenolics,-,"[AFG3, ARP6, BUD28, CAT8, CHC1, CHO1, CIK1, EC..."
4,ac_for,hydrolysates,+,"[ACF4, ADE1, ADE13, ADE17, ADE6, APA2, ARG2, A..."
5,ac_for,hydrolysates,-,"[AAC3, AIM4, AKR1, ALG9, ANP1, APQ12, AQR1, AR..."
6,fur_HMF,phenolics,+,"[MSN2, YAP1]"
7,fur_HMF,phenolics,-,"[ZWF1, ADO1, ERG3, ERG4, ERG5, BEM2, SNG1, RPL..."
8,fur_HMF,hydrolysates,+,"[HIS7, HAL5, SAT4, ADH6, CIN5, DIC1, GLR1, GSH..."
9,fur_HMF,hydrolysates,-,"[TPO3, COM2, HRK1, VID24, ZWF1, GOR1, RTG2, TR..."


### run ClueGO

In [18]:
def run_cluego(OUTPUT_FOLDER):
    #### 0.0 Start up ClueGO in case it is not running yet
    response = requests.post(CYTOSCAPE_BASE_URL+SEP+"apps"+SEP+"cluego"+SEP+"start-up-cluego", headers=HEADERS)
    # wait 2 seconds to make sure ClueGO is started
    if(str(response)=="<Response [500]>"):
        print("wait 2 secs")
        time.sleep(2) 

    #### 1.0 Select the ClueGO Organism to analyze ####
    organism_name = "Saccharomyces cerevisiae S288c" # (run "1.1 Get all ClueGO organisms" to get all options)
    print("1.0 Select the ClueGO Organism to analyze: "+organism_name)
    response = requests.put(CLUEGO_BASE_URL+SEP+"organisms"+SEP+"set-organism"+SEP+quote(organism_name), headers=HEADERS)

    ## [optional functions and settings, un comment and modify if needed]
    #  # 1.1 Get all ClueGO organisms
    #response <- requests.get(CLUEGO_BASE_URL+SEP+"organisms"+SEP+"get-all-installed-organisms")
    #print(response.json())

    # # 1.2 Get all info for installed organisms
    #response <- requests.get(CLUEGO_BASE_URL+SEP+"organisms"+SEP+"get-all-organism-info")
    #print(response.json())

    #### 2.0 Upload IDs for a specific Cluster ####
    print("2.0 Upload IDs for a specific Cluster")

    cluster1 = 1
    cluster2 = 2

    # upload gene list for cluster 1
    FILE_LOCATION = OUTPUT_FOLDER+"/gene_list.txt"
    gene_list1 = json.dumps(read_gene_list(FILE_LOCATION))
    response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"upload-ids-list"+SEP+quote(str(cluster1)), data=gene_list1, headers=HEADERS)

    # 2.1 Set the number of Clusters
    max_input_panel_number = 1
    if(EXAMPLE_SELECTION == "ClueGO Rest Example for two gene lists"):
        max_input_panel_number = 2

    response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"max-input-panel"+SEP+str(max_input_panel_number))

    if(EXAMPLE_SELECTION == "ClueGO Rest Example for two gene lists"):
        # upload gene list for cluster 2

        FILE_LOCATION = CLUEGO_HOME_FOLDER+SEP+"ClueGOExampleFiles/GSE6887_NKcell_Healthy_top200UpRegulated.txt"
        gene_list2 = json.dumps(read_gene_list(FILE_LOCATION))
        response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"upload-ids-list"+SEP+quote(str(cluster2)), data=gene_list2, headers=HEADERS)

    # To add here in the same way if you have more than 2 cluster to compare

    ## [optional functions and settings, un comment and modify if needed]
    ## 2.2 Select the ClueGO ID type
    #id_type_name = "# Automatic #" # (run "2.3 Get ClueGO ID types" to get all options)
    #response = requests.put(CLUEGO_BASE_URL+SEP+"ids"+SEP+"set-id-type"+SEP+id_type_name, headers=HEADERS)

    ## 2.3 Refresh ClueGO source files
    #requests.post(CLUEGO_BASE_URL+SEP+"ids"+SEP+"refresh-cluego-id-files", headers=HEADERS)

    ## 2.4 Get ClueGO ID types
    #response = requests.get(CLUEGO_BASE_URL+SEP+"ids"+SEP+"get-all-installed-id-types", headers=HEADERS)
    #print(response.json())

    # 2.5 Set analysis properties for a Cluster, this is to repeat for each input cluster
    input_panel_index = cluster1 # set here the cluster number e.g. "1"
    node_shape = "Ellipse" # ("Ellipse","Diamond","Hexagon","Octagon","Parallelogram","Rectangle","Round Rectangle","Triangle","V")
    cluster_color = "#ff0000" # The color in hex, e.g. #F3A455
    min_number_of_genes_per_term = 3
    min_percentage_of_genes_mapped = 4
    no_restrictions = False # "True" for no restricions in number and percentage per term
    response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"set-analysis-properties"+SEP+str(input_panel_index)+SEP+node_shape+SEP+quote(cluster_color)+SEP+str(min_number_of_genes_per_term)+SEP+str(min_percentage_of_genes_mapped)+SEP+str(no_restrictions), headers=HEADERS)

    if(EXAMPLE_SELECTION == "ClueGO Rest Example for two gene lists"):
        input_panel_index = cluster2 # set here the cluster number e.g. "2"
        node_shape = "Ellipse" # ("Ellipse","Diamond","Hexagon","Octagon","Parallelogram","Rectangle","Round Rectangle","Triangle","V")
        cluster_color = "#0000ff" # The color in hex, e.g. #F3A455
        min_number_of_genes_per_term = 3
        min_percentage_of_genes_mapped = 4
        no_restrictions = False # "True" for no restricions in number and percentage per term
        response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"set-analysis-properties"+SEP+str(input_panel_index)+SEP+node_shape+SEP+quote(cluster_color)+SEP+str(min_number_of_genes_per_term)+SEP+str(min_percentage_of_genes_mapped)+SEP+str(no_restrictions), headers=HEADERS)

    # 2.6 Select visual style
    if(EXAMPLE_SELECTION == "ClueGO Rest Example for two gene lists"):
        visual_style = "ShowClusterDifference"  # (ShowGroupDifference, ShowSignificanceDifference, ShowClusterDifference)
    else:
        visual_style = "ShowGroupDifference"  # (ShowGroupDifference, ShowSignificanceDifference, ShowClusterDifference)

    response = requests.put(CLUEGO_BASE_URL+SEP+"cluster"+SEP+"select-visual-style"+SEP+visual_style, headers=HEADERS)

    ####  3.0 Select Ontologies
    print("3.0 Select Ontologies")

    selected_ontologies = json.dumps(["3;Ellipse","5;Ellipse","7;Ellipse"]) # (run "3.1 Get all available Ontologies" to get all options)
    response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+"set-ontologies", data=selected_ontologies, headers=HEADERS)

    ## [optional functions and settings, un comment and modify if needed]
    ## 3.1 Get all available Ontologies
    #response = requests.get(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+"get-ontology-info", headers=HEADERS)
    #print(response.json())

    ## 3.2 Select Evidence Codes
    #evidence_codes = json.dumps(["All"]) # (run "3.3 Get all available Evidence Codes" to get all options)
    #response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+"set-evidence-codes", data=evidence_codes, headers=HEADERS)

    ## 3.3 Get all available Evidence Codes
    #response = requests.get(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+"get-evidence-code-info", headers=HEADERS)
    #print(response.json())

    # 3.4 Set min, max GO tree level
    min_level = 3
    max_level = 8
    all_levels = False
    response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+"set-min-max-levels"+SEP+str(min_level)+SEP+str(max_level)+SEP+str(all_levels), headers=HEADERS)

    # 3.5 Use GO term significance cutoff
    p_value_cutoff = 0.05
    use_significance_cutoff = True
    response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+str(use_significance_cutoff)+SEP+str(p_value_cutoff), headers=HEADERS)

    # 3.6 Use GO term fusion
    use_go_term_fusion = True
    response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies"+SEP+str(use_go_term_fusion), headers=HEADERS)

    # 3.7 Set statistical parameters
    enrichment_type = "Enrichment/Depletion (Two-sided hypergeometric test)" # ("Enrichment (Right-sided hypergeometric test)", "Depletion (Left-sided hypergeometric test)", "Enrichment/Depletion (Two-sided hypergeometric test)")
    multiple_testing_correction = True #Note: manually set BH-correction
    use_mid_pvalues = False
    use_doubling = False
    response = requests.put(CLUEGO_BASE_URL+SEP+"stats"+SEP+enrichment_type+SEP+str(multiple_testing_correction)+SEP+str(use_mid_pvalues)+SEP+str(use_doubling), headers=HEADERS)

    # 3.8 Set the Kappa Score level
    kappa_score = 0.4
    response = requests.put(CLUEGO_BASE_URL+SEP+"ontologies","set-kappa-score-level"+SEP+str(kappa_score), headers=HEADERS)

    # 3.9 Set grouping parameters
    do_grouping = True
    coloring_type = "Random" # ("Random","Fix")
    group_leading_term = "Highest Significance" # ("Highest Significance","#Genes / Term","%Genes / Term","%Genes / Term vs Cluster")
    grouping_type = "Kappa Score" # ("Kappa Score","Tree")
    init_group_size = 1
    perc_groups_for_merge = 50
    perc_terms_for_merge = 50
    response = requests.put(CLUEGO_BASE_URL+SEP+"grouping"+SEP+str(do_grouping)+SEP+coloring_type+SEP+group_leading_term+SEP+grouping_type+SEP+str(init_group_size)+SEP+str(perc_groups_for_merge)+SEP+str(perc_terms_for_merge), headers=HEADERS)

    #### 4.0 Run ClueGO Analysis ####
    print("4.0 Run ClueGO Analysis")

    # Run the analysis an save log file
    analysis_name = EXAMPLE_SELECTION
    analysis_option = "Cancel and refine selection" # ("Continue analysis","Skip the grouping","Cancel and refine selection")  -> Analysis option in case there are more than 1000 terms found!
    response = requests.get(CLUEGO_BASE_URL+SEP+quote(analysis_name)+SEP+quote(analysis_option), headers=HEADERS)
    log_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Example-Analysis-log.txt"
    writeLines(response.text,log_file_name)
    #print(response.text)

    # 4.1 Get network id (SUID) (CyRest function from Cytoscape)
    response = requests.get(CYTOSCAPE_BASE_URL+SEP+"networks"+SEP+"currentNetwork", headers=HEADERS)
    current_network_suid = response.json()['data']['networkSUID']
    #print(current_network_suid)

    print("Save results")

    # Get network graphics (CyRest function from Cytoscape)
    image_type = "svg" # png, pdf
    response = requests.get(CYTOSCAPE_BASE_URL+SEP+"networks"+SEP+str(current_network_suid)+SEP+"views"+SEP+"first."+image_type)
    image_file_name = OUTPUT_FOLDER+SEP+"ClueGOExampleNetwork."+image_type
    writeBin(response.content,image_file_name)

    # 4.2 Get ClueGO result table
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-cluego-table"+SEP+str(current_network_suid))
    table_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Example-Result-Table.txt"
    writeLines(response.text,table_file_name)

    # 4.3 Get ClueGO genes and main functions
    number_of_functions_to_add = 3
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-main-functions"+SEP+str(current_network_suid)+SEP+str(number_of_functions_to_add))
    table_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Genes-With-Main-Functions.txt"
    writeLines(response.text,table_file_name)

    # 4.4 Get genes result table
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-gene-table"+SEP+str(current_network_suid))
    table_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Gene-Table.txt"
    writeLines(response.text,table_file_name)

    # 4.5 Get Kappascore Matrix
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-kappascore-matrix"+SEP+str(current_network_suid))
    table_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Kappascore-Matrix.txt"
    writeLines(response.text,table_file_name)

    # 4.6 Get binary Gene-Term Matrix
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-binary-gene-term-matrix"+SEP+str(current_network_suid))
    table_file_name = OUTPUT_FOLDER+SEP+"ClueGO-Binary-Gene-Term-Matrix.txt"
    writeLines(response.text,table_file_name)

    # 4.7 ClueGO Result Chart
    # Get result charts for both cluster as pie chart
    chart_type = "PieChart" # ("PieChart","BarChart")
    image_type = "svg" # ("svg","png","pdf")
    response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-cluego-result-chart"+SEP+str(current_network_suid)+SEP+str(cluster1)+SEP+chart_type+SEP+image_type)
    image_file_name = OUTPUT_FOLDER+SEP+"ClueGO-"+chart_type+"-For-Cluster"+str(cluster1)+"."+image_type
    writeBin(response.content,image_file_name)

    if(EXAMPLE_SELECTION == "ClueGO Rest Example for two gene lists"):
        response = requests.get(CLUEGO_BASE_URL+SEP+"analysis-results"+SEP+"get-cluego-result-chart"+SEP+str(current_network_suid)+SEP+str(cluster2)+SEP+chart_type+SEP+image_type)
        image_file_name = OUTPUT_FOLDER+SEP+"ClueGO-"+chart_type+"-For-Cluster"+str(cluster2)+"."+image_type
        writeBin(response.content,image_file_name)

    ### [optional functions and settings, un comment and modify if needed]
    ## 4.8 Remove ClueGO analysis result
    #print("Remove ClueGO Network")
    ## Remove analysis to reduce memory usage. This is important when using patch modes that create lots of analyses.
    #response = requests.delete(CLUEGO_BASE_URL+SEP+"remove-cluego-analysis-result"+SEP+str(current_network_suid))

    ## 4.9 Remove all ClueGO analysis results
    #print("Remove all ClueGO Networks")
    #response = requests.delete(CLUEGO_BASE_URL+SEP+"remove-all-cluego-analysis-results")

    print("Example analysis "+EXAMPLE_SELECTION+" done")


In [19]:
for idx,row in gene_lists.iterrows():
    OUTPUT_FOLDER2 = OUTPUT_FOLDER+'/'+row.inh1+'_'+row.inh2+'_'+row.effect
    !mkdir {OUTPUT_FOLDER2}
    print(OUTPUT_FOLDER2)
    
    # save to list of genes newline
    with open(OUTPUT_FOLDER2+"/gene_list.txt", "w") as f:
        for i in row.names:
            f.write(i+'\n')
            
    run_cluego(OUTPUT_FOLDER2)


mkdir: /Users/zrimec/ClueGO_output/ac_for_fur_HMF_+: File exists
/Users/zrimec/ClueGO_output/ac_for_fur_HMF_+
1.0 Select the ClueGO Organism to analyze: Saccharomyces cerevisiae S288c
2.0 Upload IDs for a specific Cluster
3.0 Select Ontologies
4.0 Run ClueGO Analysis
Save results
Example analysis ClueGO_output done
mkdir: /Users/zrimec/ClueGO_output/ac_for_fur_HMF_-: File exists
/Users/zrimec/ClueGO_output/ac_for_fur_HMF_-
1.0 Select the ClueGO Organism to analyze: Saccharomyces cerevisiae S288c
2.0 Upload IDs for a specific Cluster
3.0 Select Ontologies
4.0 Run ClueGO Analysis
Save results
Example analysis ClueGO_output done
mkdir: /Users/zrimec/ClueGO_output/ac_for_phenolics_+: File exists
/Users/zrimec/ClueGO_output/ac_for_phenolics_+
1.0 Select the ClueGO Organism to analyze: Saccharomyces cerevisiae S288c
2.0 Upload IDs for a specific Cluster
3.0 Select Ontologies
4.0 Run ClueGO Analysis
Save results
Example analysis ClueGO_output done
mkdir: /Users/zrimec/ClueGO_output/ac_for_phe